## Import Libraries

In [79]:
import pandas as pd
import numpy as np
import os
import librosa 
import parselmouth as pm
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
import torch

## Create Features for DataFrame

In [27]:
paths = []
labels = []

In [28]:
audio_path = "F:\Final Datatset\Final Datatset"
for filename in os.listdir(audio_path):
    if filename.endswith(".wav") or file_name.endswith(".mp3"):  # Add other extensions if needed
        paths.append(os.path.join(audio_path, filename))
        label = filename.split("_")[-1]
        label = label.split(".")[0]
        labels.append(label.lower())
        #print(filename)

In [29]:
paths[:5]

['F:\\Final Datatset\\Final Datatset\\OAF_back_angry.wav',
 'F:\\Final Datatset\\Final Datatset\\OAF_back_disgust.wav',
 'F:\\Final Datatset\\Final Datatset\\OAF_back_fear.wav',
 'F:\\Final Datatset\\Final Datatset\\OAF_back_happy.wav',
 'F:\\Final Datatset\\Final Datatset\\OAF_back_neutral.wav']

In [30]:
labels[:5]

['angry', 'disgust', 'fear', 'happy', 'neutral']

In [31]:
import os
import librosa
import numpy as np
import parselmouth

data = []
pitch_aud = []
loudness_aud = []  # intensity
rhythm_aud = []
zcr_aud = []
mfcc_aud = []
rmse_aud = []

i = 1

def pitch(audio_file):
    # Load audio using librosa
    y, sr = librosa.load(audio_file, sr=None)
    # Extract pitch using piptrack
    pitches, magnitudes = librosa.piptrack(y=y, sr=sr)
    return np.squeeze(pitches)

def loudness(audio_file):
    # Load audio using parselmouth
    snd = parselmouth.Sound(audio_file)
    # Extract intensity (loudness)
    intensity = snd.to_intensity()
    return np.squeeze(intensity.values )  # Extracting the intensity values

def rhythm(audio_file):
    intensity_values = loudness(audio_file)
    rhythm_value = np.var(intensity_values) if len(intensity_values) > 0 else 0
    return rhythm_value

def zcr(audio_file):
    y, sr = librosa.load(audio_file, sr=None)
    zcr=librosa.feature.zero_crossing_rate(y)
    return np.squeeze(zcr)

def mfcc_extract(audio_file):
    global i
    y, sr = librosa.load(audio_file, sr=None)
    mfcc=np.mean(librosa.feature.mfcc(y=y,sr=sr,n_mfcc=13).T,axis=0)
    return mfcc

def rmse(audio_file):
    y, sr = librosa.load(audio_file, sr=None)
    rmse=librosa.feature.rms(y=y)
    return np.squeeze(rmse)

# Use raw string to avoid path issues
directory = r"F:\Final Datatset\Final Datatset"

for filename in os.listdir(directory):
    if filename.endswith('.wav'):
        audio_path = os.path.join(directory, filename)
        try:
            pitch_aud.append(pitch(audio_path))
            loudness_aud.append(loudness(audio_path))
            rhythm_aud.append(rhythm(audio_path))
            zcr_aud.append(zcr(audio_path))
            mfcc_aud.append(mfcc_extract(audio_path))
            rmse_aud.append(rmse(audio_path))
        except Exception as e:
            print(f"Error processing {audio_path}: {e}")


In [32]:
for i in range(len(labels)):
    audio_data = {
        "pitch_audio":pitch_aud[i],
        "loudness_aud":loudness_aud[i],
        "rhythm_aud":rhythm_aud[i],
        "zcr_aud":zcr_aud[i],
        "mfcc_aud":mfcc_aud[i],
        "rmse_aud":rmse_aud[i],
        "label":labels[i],
        "Name":paths[i]
    }
    data.append(audio_data)

print(len(data))

1400


In [33]:
df = pd.DataFrame(data)
df.head()

,pitch_audio,loudness_aud,rhythm_aud,zcr_aud,mfcc_aud,rmse_aud,label,Name
0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[38.03750304214155, 41.49734917689817, 43.7337...",164.310292,"[0.00244140625, 0.08447265625, 0.19384765625, ...","[-390.03134, 55.235966, -15.03465, -10.704581,...","[0.0010797889, 0.0020301517, 0.0035413173, 0.0...",angry,F:\Final Datatset\Final Datatset\OAF_back_angr...
1,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[38.996866542674454, 40.6122316619878, 39.4057...",63.815818,"[0.0263671875, 0.078125, 0.14404296875, 0.2695...","[-486.6104, 90.02398, 13.257611, -4.3464375, 1...","[0.0011371016, 0.0014314997, 0.0018551903, 0.0...",disgust,F:\Final Datatset\Final Datatset\OAF_back_disg...
2,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[25.447448217789955, 27.80412910776463, 32.757...",181.433429,"[0.00048828125, 0.00634765625, 0.10009765625, ...","[-437.2444, 60.60631, 4.735497, -8.189329, 1.7...","[0.0016193774, 0.0018962268, 0.002077534, 0.00...",fear,F:\Final Datatset\Final Datatset\OAF_back_fear...
3,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[42.22079079128535, 43.55058698637102, 44.6725...",116.410430,"[0.146484375, 0.279296875, 0.416015625, 0.4721...","[-471.816, 83.897, 14.153293, -11.087668, 1.56...","[0.0015981119, 0.0023980488, 0.003265329, 0.00...",happy,F:\Final Datatset\Final Datatset\OAF_back_happ...
4,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[36.67417807124207, 34.740853139384306, 32.572...",68.954396,"[0.00341796875, 0.005859375, 0.0068359375, 0.0...","[-528.7869, 85.98792, 17.729805, 1.2786299, 14...","[0.0006581845, 0.0009542859, 0.0013058955, 0.0...",neutral,F:\Final Datatset\Final Datatset\OAF_back_neut...


In [46]:
X = df["mfcc_aud"]

In [47]:
x = [X for X in X]
x = np.array(x)
x.shape

(1400, 13)

In [51]:
print(y.shape)
print(x)
print(x.dtype)
print(y.dtype)

(1400, 7)
[[-390.03134      55.235966    -15.03465    ...    1.646008
    -0.94095045    2.2132027 ]
 [-486.6104       90.02398      13.257611   ...   -5.090566
    -3.4799967     3.4800596 ]
 [-437.2444       60.60631       4.735497   ...   -6.0952525
   -12.106713     -2.660847  ]
 ...
 [-408.7838       69.757454     23.994398   ...  -15.896726
   -13.018727     -5.769766  ]
 [-538.86615      83.389206     31.602024   ...   -8.412885
    -2.8169267    -2.8199797 ]
 [-529.4634       68.79754      16.836895   ...   -9.885112
    -2.917587     -0.94027686]]
float32
float64


In [52]:
df_train = pd.DataFrame(x, columns=[f"mfcc {i+1}" for i in range(13)])
df_train.head()

,mfcc 1,mfcc 2,mfcc 3,mfcc 4,mfcc 5,mfcc 6,mfcc 7,mfcc 8,mfcc 9,mfcc 10,mfcc 11,mfcc 12,mfcc 13
0,-390.031342,55.235966,-15.034650,-10.704581,-6.318040,9.424742,-20.741861,-12.001622,-7.151769,-7.919952,1.646008,-0.940950,2.213203
1,-486.610413,90.023979,13.257611,-4.346437,11.698466,7.128137,-6.389263,1.944248,-13.579486,-0.372804,-5.090566,-3.479997,3.480060
2,-437.244385,60.606312,4.735497,-8.189329,1.751806,5.401639,-12.745217,-1.665546,-9.431613,0.366829,-6.095253,-12.106713,-2.660847
3,-471.816010,83.897003,14.153293,-11.087668,1.563715,10.341300,-12.095528,0.680853,-5.472232,-3.928490,-4.276334,-10.476326,-1.465433
4,-528.786926,85.987923,17.729805,1.278630,14.709511,12.722618,-4.685200,-4.697146,-10.850562,2.226550,-3.049066,-2.551708,0.782543


In [53]:
df_train['label'] = df['label'].values
df_train.head()

,mfcc 1,mfcc 2,mfcc 3,mfcc 4,mfcc 5,mfcc 6,mfcc 7,mfcc 8,mfcc 9,mfcc 10,mfcc 11,mfcc 12,mfcc 13,label
0,-390.031342,55.235966,-15.034650,-10.704581,-6.318040,9.424742,-20.741861,-12.001622,-7.151769,-7.919952,1.646008,-0.940950,2.213203,angry
1,-486.610413,90.023979,13.257611,-4.346437,11.698466,7.128137,-6.389263,1.944248,-13.579486,-0.372804,-5.090566,-3.479997,3.480060,disgust
2,-437.244385,60.606312,4.735497,-8.189329,1.751806,5.401639,-12.745217,-1.665546,-9.431613,0.366829,-6.095253,-12.106713,-2.660847,fear
3,-471.816010,83.897003,14.153293,-11.087668,1.563715,10.341300,-12.095528,0.680853,-5.472232,-3.928490,-4.276334,-10.476326,-1.465433,happy
4,-528.786926,85.987923,17.729805,1.278630,14.709511,12.722618,-4.685200,-4.697146,-10.850562,2.226550,-3.049066,-2.551708,0.782543,neutral


In [75]:
x = df_train[[f"mfcc {i+1}" for i in range(13)]]
y = df_train['label']

In [76]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [77]:
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.fit_transform(y_test)

In [103]:
X_train = X_train.astype(float)
X_test = X_test.astype(float)

X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)  
y_train_tensor = torch.tensor(y_train, dtype=torch.long)  
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)  
y_test_tensor = torch.tensor(y_test, dtype=torch.long)  

# Print shapes to verify correctness
print(f"X_train_tensor shape: {X_train_tensor.shape}, y_train_tensor shape: {y_train_tensor.shape}")
print(f"X_test_tensor shape: {X_test_tensor.shape}, y_test_tensor shape: {y_test_tensor.shape}")


X_train_tensor shape: torch.Size([1120, 13]), y_train_tensor shape: torch.Size([1120])
X_test_tensor shape: torch.Size([280, 13]), y_test_tensor shape: torch.Size([280])


In [ ]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

class MyModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(MyModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = x.unsqueeze(1) 
        _, (hn, _) = self.lstm(x)  
        out = self.fc(hn[-1])  
        return out

In [ ]:
train_dataset = TensorDataset(X_train_tensor,y_train_tensor)
train_dataloader = DataLoader(train_dataset,  batch_size=128, shuffle=True)

# Model hyperparameters
input_size = 13  
hidden_size = 128  
num_classes = len(label_encoder.classes_)

In [106]:
model = MyModel(input_size, hidden_size, num_classes)
criterion = nn.CrossEntropyLoss()  # Loss function for classification tasks
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Optimizer

In [114]:
epochs = 100
for epoch in range(epochs):
    model.train()  # Set the model to training mode
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    
    optimizer.zero_grad()  # Zero the gradient buffers
    loss.backward()  # Backpropagate the loss
    optimizer.step()  # Update the weights
    
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation (using test data, replace with real test data)
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    y_pred = model(X_train_tensor)  # Using the training tensor for demonstration
    y_pred_classes = torch.argmax(y_pred, dim=1).numpy()
    y_true = y_train_tensor.numpy()
    print("Predicted classes:", y_pred_classes)
    print("Shape of y_true:", len(y_true))
    print("Shape of y_pred:", len(y_pred))
    print(classification_report(y_true, y_pred_classes, target_names=label_encoder.classes_))

Epoch [10/100], Loss: 0.1143
Epoch [20/100], Loss: 0.1128
Epoch [30/100], Loss: 0.1113
Epoch [40/100], Loss: 0.1098
Epoch [50/100], Loss: 0.1083
Epoch [60/100], Loss: 0.1069
Epoch [70/100], Loss: 0.1055
Epoch [80/100], Loss: 0.1042
Epoch [90/100], Loss: 0.1028
Epoch [100/100], Loss: 0.1015
Predicted classes: [2 0 2 ... 0 0 0]
Shape of y_true: 1120
Shape of y_pred: 1120
              precision    recall  f1-score   support

       angry       1.00      1.00      1.00       166
     disgust       0.99      0.98      0.98       145
        fear       0.98      0.97      0.98       156
       happy       0.94      0.99      0.96       154
     neutral       0.95      0.99      0.97       159
          ps       0.98      0.93      0.96       166
         sad       0.99      0.97      0.98       174

    accuracy                           0.97      1120
   macro avg       0.98      0.98      0.97      1120
weighted avg       0.98      0.97      0.98      1120

